In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras.optimizers import Adam
from keras.regularizers import l2
from kerastuner.tuners import RandomSearch

C:\Users\andre\AppData\Local\Temp\ipykernel_11496\967855174.py:11: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [2]:
# Load the dataset
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
column_names = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome']
dataset = pd.read_csv(url, header=None, names=column_names)

# Separate features and target
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

# Standardize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)


In [3]:
def build_model(hp):
    model = Sequential()
    model.add(Dense(hp.Int('units1', min_value=64, max_value=512, step=32), input_shape=(X.shape[1],),
                    activation='relu', kernel_regularizer=l2(0.001)))
    model.add(BatchNormalization())
    model.add(Dropout(hp.Float('dropout1', 0.1, 0.5, step=0.1)))

    model.add(Dense(hp.Int('units2', min_value=64, max_value=512, step=32), activation='relu',
                    kernel_regularizer=l2(0.001)))
    model.add(BatchNormalization())
    model.add(Dropout(hp.Float('dropout2', 0.1, 0.5, step=0.1)))

    model.add(Dense(hp.Int('units3', min_value=64, max_value=512, step=32), activation='relu',
                    kernel_regularizer=l2(0.001)))
    model.add(BatchNormalization())
    model.add(Dropout(hp.Float('dropout3', 0.1, 0.5, step=0.1)))

    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer=Adam(hp.Float('learning_rate', 1e-5, 1e-2, sampling='LOG')),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [4]:
# Initialize Keras Tuner
tuner = RandomSearch(build_model,
                     objective='val_accuracy',
                     max_trials=30,
                     executions_per_trial=3,
                     directory='my_dir',
                     project_name='pima_diabetes')

Reloading Tuner from my_dir\pima_diabetes\tuner0.json


In [5]:
# Prepare K-Fold Cross-Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Lists to store results from each fold
fold_accuracies = []


In [6]:
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Search for the best hyperparameters
    tuner.search(X_train, y_train, epochs=100, validation_data=(X_test, y_test),
                 callbacks=[EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)])

    # Get the optimal hyperparameters
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

    # Build the model with the optimal hyperparameters
    model = tuner.hypermodel.build(best_hps)

    # Define callbacks
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)
    checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)

    # Train the model with the optimal hyperparameters
    model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test),
              callbacks=[early_stopping, reduce_lr, checkpoint])

    # Load the best model
    model.load_weights('best_model.h5')

    # Evaluate the model
    y_pred = (model.predict(X_test) > 0.5).astype("int32")
    accuracy = accuracy_score(y_test, y_pred)
    fold_accuracies.append(accuracy)

Trial 30 Complete [00h 00m 30s]
val_accuracy: 0.7835497657457987

Best val_accuracy So Far: 0.8116883039474487
Total elapsed time: 01h 16m 08s
Epoch 1/100
20/20 [==============================] - 2s 20ms/step - loss: 1.2502 - accuracy: 0.6775 - val_loss: 1.0512 - val_accuracy: 0.7338 - lr: 0.0042
Epoch 2/100
11/20 [===============>..............] - ETA: 0s - loss: 1.0133 - accuracy: 0.7699

c:\Users\andre\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


20/20 [==============================] - 0s 9ms/step - loss: 1.0354 - accuracy: 0.7541 - val_loss: 1.0414 - val_accuracy: 0.7208 - lr: 0.0042
Epoch 3/100
20/20 [==============================] - 0s 12ms/step - loss: 1.0142 - accuracy: 0.7622 - val_loss: 1.0295 - val_accuracy: 0.7468 - lr: 0.0042
Epoch 4/100
20/20 [==============================] - 0s 7ms/step - loss: 0.9943 - accuracy: 0.7345 - val_loss: 1.0373 - val_accuracy: 0.7208 - lr: 0.0042
Epoch 5/100
20/20 [==============================] - 0s 11ms/step - loss: 0.9627 - accuracy: 0.7606 - val_loss: 0.9985 - val_accuracy: 0.7468 - lr: 0.0042
Epoch 6/100
20/20 [==============================] - 0s 10ms/step - loss: 0.9042 - accuracy: 0.7720 - val_loss: 0.9982 - val_accuracy: 0.7532 - lr: 0.0042
Epoch 7/100
20/20 [==============================] - 0s 12ms/step - loss: 0.8853 - accuracy: 0.7622 - val_loss: 0.9607 - val_accuracy: 0.7403 - lr: 0.0042
Epoch 8/100
20/20 [==============================] - 0s 13ms/step - loss: 0.8496 - a

c:\Users\andre\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


20/20 [==============================] - 0s 8ms/step - loss: 1.0930 - accuracy: 0.7443 - val_loss: 1.0276 - val_accuracy: 0.7208 - lr: 0.0042
Epoch 3/100
20/20 [==============================] - 0s 11ms/step - loss: 1.0289 - accuracy: 0.7296 - val_loss: 0.9960 - val_accuracy: 0.7208 - lr: 0.0042
Epoch 4/100
20/20 [==============================] - 0s 12ms/step - loss: 0.9342 - accuracy: 0.7785 - val_loss: 0.9766 - val_accuracy: 0.7273 - lr: 0.0042
Epoch 5/100
20/20 [==============================] - 0s 9ms/step - loss: 0.9357 - accuracy: 0.7704 - val_loss: 0.9602 - val_accuracy: 0.7078 - lr: 0.0042
Epoch 6/100
20/20 [==============================] - 0s 12ms/step - loss: 0.8812 - accuracy: 0.7687 - val_loss: 0.9354 - val_accuracy: 0.7013 - lr: 0.0042
Epoch 7/100
20/20 [==============================] - 0s 12ms/step - loss: 0.8693 - accuracy: 0.7606 - val_loss: 0.8961 - val_accuracy: 0.7273 - lr: 0.0042
Epoch 8/100
20/20 [==============================] - 0s 12ms/step - loss: 0.8480 - a

...

20/20 [==============================] - 0s 7ms/step - loss: 1.0337 - accuracy: 0.7443 - val_loss: 1.1251 - val_accuracy: 0.6364 - lr: 0.0042
Epoch 3/100
20/20 [==============================] - 0s 11ms/step - loss: 1.0132 - accuracy: 0.7606 - val_loss: 1.0615 - val_accuracy: 0.6688 - lr: 0.0042
Epoch 4/100
20/20 [==============================] - 0s 9ms/step - loss: 0.9318 - accuracy: 0.7883 - val_loss: 1.0224 - val_accuracy: 0.6883 - lr: 0.0042
Epoch 5/100
20/20 [==============================] - 0s 11ms/step - loss: 0.9500 - accuracy: 0.7524 - val_loss: 1.0126 - val_accuracy: 0.6558 - lr: 0.0042
Epoch 6/100
20/20 [==============================] - 0s 11ms/step - loss: 0.9191 - accuracy: 0.7573 - val_loss: 0.9629 - val_accuracy: 0.6883 - lr: 0.0042
Epoch 7/100
20/20 [==============================] - 0s 11ms/step - loss: 0.8683 - accuracy: 0.7834 - val_loss: 0.9476 - val_accuracy: 0.6753 - lr: 0.0042
Epoch 8/100
20/20 [==============================] - 0s 11ms/step - loss: 0.8251 - a

c:\Users\andre\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 3/100
20/20 [==============================] - 0s 14ms/step - loss: 0.9979 - accuracy: 0.7577 - val_loss: 0.9862 - val_accuracy: 0.7647 - lr: 0.0042
Epoch 4/100
20/20 [==============================] - 0s 9ms/step - loss: 0.9495 - accuracy: 0.7707 - val_loss: 1.0062 - val_accuracy: 0.7386 - lr: 0.0042
Epoch 5/100
20/20 [==============================] - 0s 13ms/step - loss: 0.9507 - accuracy: 0.7593 - val_loss: 0.9556 - val_accuracy: 0.7451 - lr: 0.0042
Epoch 6/100
20/20 [==============================] - 0s 9ms/step - loss: 0.9000 - accuracy: 0.7707 - val_loss: 0.9556 - val_accuracy: 0.7451 - lr: 0.0042
Epoch 7/100
20/20 [==============================] - 0s 12ms/step - loss: 0.8516 - accuracy: 0.7772 - val_loss: 0.9057 - val_accuracy: 0.7386 - lr: 0.0042
Epoch 8/100
20/20 [==============================] - 0s 12ms/step - loss: 0.8274 - accuracy: 0.7919 - val_loss: 0.8932 - val_accuracy: 0.7386 - lr: 0.0042
Epoch 9/100
20/20 [==============================] - 0s 12ms/step - loss

c:\Users\andre\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


20/20 [==============================] - 0s 10ms/step - loss: 1.0989 - accuracy: 0.7350 - val_loss: 1.0677 - val_accuracy: 0.6732 - lr: 0.0042
Epoch 3/100
20/20 [==============================] - 0s 13ms/step - loss: 1.0535 - accuracy: 0.7512 - val_loss: 1.0461 - val_accuracy: 0.6797 - lr: 0.0042
Epoch 4/100
20/20 [==============================] - 0s 12ms/step - loss: 0.9991 - accuracy: 0.7642 - val_loss: 1.0021 - val_accuracy: 0.7320 - lr: 0.0042
Epoch 5/100
20/20 [==============================] - 0s 13ms/step - loss: 0.9460 - accuracy: 0.7740 - val_loss: 0.9873 - val_accuracy: 0.7190 - lr: 0.0042
Epoch 6/100
20/20 [==============================] - 0s 13ms/step - loss: 0.9479 - accuracy: 0.7480 - val_loss: 0.9709 - val_accuracy: 0.7059 - lr: 0.0042
Epoch 7/100
20/20 [==============================] - 0s 13ms/step - loss: 0.9020 - accuracy: 0.7886 - val_loss: 0.9347 - val_accuracy: 0.7320 - lr: 0.0042
Epoch 8/100
20/20 [==============================] - 0s 13ms/step - loss: 0.8583 -

In [7]:
# Calculate the mean accuracy across all folds
mean_accuracy = np.mean(fold_accuracies)
print(f'Mean Accuracy across all folds: {mean_accuracy}')

# Save the final model
model.save('diabetters.h5')
print("Final optimized model saved as 'diabetters.h5'")


Mean Accuracy across all folds: 0.7616670910788559
Final optimized model saved as 'diabetters.h5'


c:\Users\andre\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [10]:
# Load the saved model
from keras.models import load_model

# Load the final optimized model
final_model = load_model('diabetters.h5')

# Example input data for prediction (same preprocessing as training data)
example_input = np.array([[11,138,76,0,0,33.2,0.42,35]])
example_input = scaler.transform(example_input)

# Predict the outcome
example_prediction = final_model.predict(example_input)
predicted_class = (example_prediction > 0.5).astype("int32")

print(f'Predicted class for the example input: {predicted_class[0][0]}')

1/1 [==============================] - 0s 102ms/step
Predicted class for the example input: 1
